In [17]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.duca.manual_seed_all(777)

In [19]:
#parameters
learning_rate = 0.001
training_epochs =15
batch_size = 100


In [20]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                         transform = transforms.ToTensor(),
                         download =True)
mnist_test = dests.MNIST(root = "MNIST_data/",
                        train= False,
                        transform =transforms.ToTensor(),
                        download =True)

Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw


Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw


Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw


Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw
Processing...


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [22]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                            drop_last=True
                                          )


In [25]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1= nn.Sequential(
        nn.Conv2d(1,32,kernel_size= 3 , stride =1, padding =1),
        nn.ReLUs(),
        nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(32,64, kernel_size= 3, stride =1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.fc= nn.Linear(7*7*64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out

In [26]:
model = CNN().to(device)


In [28]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [1]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device) #이미지
        Y = Y.to(device ) # label
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost /total_batch
    print('[Epoch : {}] cost = {}'.format(epoch+1, avg_cost))

NameError: name 'data_loader' is not defined

In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuray: ',accuracy.Item() )

ERROR: Could not find a version that satisfies the requirement visdom.server (from versions: none)
ERROR: No matching distribution found for visdom.server
